<a href="https://colab.research.google.com/github/DanielKorenDataScience/NLP-FinalProject/blob/main/ChatBotBase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from importlib import import_module
def my_import(lib, package=None, func=None):
  try:
    import_module(lib)
  except:
    if package is None: package = lib
    !pip install "{package}"
    import_module(lib)
  
  if func is not None:
    return import_module(lib).__getattribute__(func)

In [ ]:
my_import('pyforest')

p = print
d = display

  Created wheel for pyforest: filename=pyforest-1.1.0-py2.py3-none-any.whl size=14606 sha256=146c2f0d4c4f0b08c58d4141ea62c34484a3c2ff140521e51ff0d92a8a245627
  Stored in directory: /root/.cache/pip/wheels/40/c6/da/43562aeea85b37f1a2b3d326f0f602f865000d2ada8a43625f
Successfully built pyforest


In [ ]:
my_import('spacy')
import requests

api_key = 'c2adfa29edfd95ad16efab9218619ff3'
URL = "http://api.openweathermap.org/data/2.5/weather?"

# Load md (medium) English tokenizer, tagger, parser and NER, and vectors(vectors are not included in the 'sm' model)
#nlp = spacy.load('en_core_web_md')  NOT WORKING ON COLAB
try:
  import en_core_web_md
except:  
  !python -m spacy download en_core_web_md
  import en_core_web_md
  
nlp = en_core_web_md.load()

     |████████████████████████████████| 96.4MB 1.3MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp37-none-any.whl size=98051304 sha256=9f39d24f01b28abf08c375d69084e2bda97d53e433fc3ee8debadddb6f058e23
  Stored in directory: /tmp/pip-ephem-wheel-cache-rr9mgybr/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
TEST = True

In [ ]:
def get_weather(city_name):
    api_url = URL + "q={}&appid={}".format(city_name, api_key)

    response = requests.get(api_url)
    response_dict = response.json()

    weather = response_dict#["weather"]#[0]#["description"]

    if response.status_code == 200:
        return weather
    else:
        print('[!] HTTP {0} calling [{1}]'.format(response.status_code, api_url))
        return None

if TEST:
  weather = get_weather("London")
  print(weather)        

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 287.66, 'feels_like': 287.62, 'temp_min': 285.3, 'temp_max': 290.03, 'pressure': 1012, 'humidity': 94}, 'visibility': 6000, 'wind': {'speed': 2.57, 'deg': 10}, 'clouds': {'all': 90}, 'dt': 1624018969, 'sys': {'type': 2, 'id': 268730, 'country': 'GB', 'sunrise': 1623987763, 'sunset': 1624047637}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


In [ ]:
WEATHER_ONLY = 1

In [ ]:
def chatbot_response(statement):
  template = "Current weather in a city"
  doc = nlp(statement)
  min_similarity = 0.75

  #similarity = calculate_similarity(statement, template)
  #p(similarity)
  similarity = 1

  if len(doc) == 1 or similarity >= min_similarity:
    city = ''
    for ent in doc.ents:
      if ent.label_ == "GPE": # GeoPolitical Entity
        city = ent.text
        break
      else:
        return "You need to tell me a city to check."
    if city == '':
      return "You need to tell me a city to check."

    template = WEATHER_ONLY

    city_weather = get_weather(city)
    if city_weather is not None:
      if template == WEATHER_ONLY:
        result = city_weather["weather"][0]["description"]
      return f"In {city}, the current weather is: {result}"
    else:
      return "Something went wrong."
  else:
    return "Sorry I don't understand that. Please rephrase your statement."

chatbot_response('Current Weather in Cleveland')
#chatbot_response('tel aviv')

'In Cleveland, the current weather is: broken clouds'

In [ ]:
def get_parts(text, disp=False):
  doc = nlp(text.lower())
  date, gpe = [], []
  action = ''
  p()
  if disp:
    for token in doc:
      p(token.text, token.dep_, token.ent_type_, token.head, token.lemma_, token.pos_, token.tag_)
  for entity in doc.ents:
      if disp:
        print(entity.text, entity.label_)
      if entity.label_ == 'DATE': date.append(entity.text.lower())
      if entity.label_ == 'GPE': gpe.append(entity.text.lower())
  if len(date) == 0:
    date = ['current']
  p(date, gpe)

In [ ]:
get_parts("Today weather in a Ra-anana")
get_parts("weather paris next 5 days")
get_parts("weather paris today and tomorrow")
get_parts("weather paris")
get_parts("weather paris, canada")
get_parts("weather paris Canada", True)
get_parts("weather paris in December")
get_parts("weather in tel aviv")
get_parts("tomorrow weather in haifa")
get_parts("tomorrow rain in haifa", True)


['today'] []

['next 5 days'] ['paris']

['today', 'tomorrow'] ['paris']

['current'] ['paris']

['current'] ['paris', 'canada']

weather compound  canada weather PROPN NNP
paris compound  canada paris PROPN NNP
canada ROOT  canada canada PROPN NNP
['current'] []

['december'] ['paris']

['current'] ['tel aviv']

['tomorrow'] ['haifa']

tomorrow compound DATE rain tomorrow NOUN NN
rain ROOT  rain rain NOUN NN
in prep  rain in ADP IN
haifa pobj GPE in haifa PROPN NNP
tomorrow DATE
haifa GPE
['tomorrow'] ['haifa']


In [ ]:
while True:
  msg = input()
  if msg == '': break
  p(chatbot_response(msg))
  p()

weather in hadera
In hadera, the current weather is: clear sky


